In [1]:
import requests
import os
import base64
import csv
import json
import re
import pandas as pd
from datetime import datetime
from spotify_client import *
from credentials.config import *

In [2]:
def refresh_accesss_token():
    client_creds = f'{client_id}:{client_secret}'
    client_creds_b64 = base64.b64encode(client_creds.encode())

    refresh_token_header = {
        'Authorization' : f'Basic {client_creds_b64.decode()}'
    }

    # def refresh_playlist_token():
    refresh_url =  'https://accounts.spotify.com/api/token'
    refresh_params = {
        'grant_type': 'refresh_token',
        'refresh_token': f'{refresh_token}'    
    }
    r_token = requests.post(refresh_url, data=refresh_params, headers=refresh_token_header)
    refresh_response = r_token.json()

    access_token = refresh_response['access_token']

    return access_token

In [4]:
playlist_token = refresh_accesss_token()

###
# imported into tkinter as a default entry in for the scrape button
# left here in case using the scrape without the gui
# replace all cases of week_num with this funciton variable: get_week_num()
###

In [5]:
def get_track_features(track_uris):
    playlist_token = refresh_accesss_token()
    track_ids = [track[14:] for track in track_uris]
    track_limit = 100 
    # using list comprehension 
    batched_tracks = [track_ids[i * track_limit:(i + 1) * track_limit] for i in range((len(track_ids) + track_limit - 1) // track_limit)]  
#     url = f'https://api.spotify.com/v1/audio-features?ids='
#     =7ouMYWpwJ422jRcDASZB7P%2C4VqPOruhp5EdPBeR92t6lQ%2C2takcwOaAZWiXQijPHIx7B'
    for batch in batched_tracks:
        request_data =  "%2C".join(batch)
        
        url = f'https://api.spotify.com/v1/audio-features?ids={request_data}'
        response = requests.get(
            url,
            headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
        # print(response.url)
        b = response.json()
        return print(b)

In [6]:
def get_album_tracks(album_ids):
    track_uris = []
#     request_data = json.dumps(uris)
    for album_id in album_ids:
        query = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
        response = requests.get(
            query,
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {playlist_token}'
            })
        response_json = response.json()
        # print(response_json)
        tracks = response_json['items']
        for track in tracks:
            track_uris.append(track['uri'])
    get_track_features(track_uris)
    return track_uris

In [12]:

# def search_for_albums(csv_path):
artists=[]
albums=[]

albums_not_found = {'artist': [], 'album': []}
album_ids = set()
album_list = []
list_ids = []

meta_16_df = pd.read_csv('albums_2016.csv')

no_nan_df = meta_16_df.dropna()
# no_nan_df.info()
#   initialize spotify client
spotify = SpotifyAPI(client_id, client_secret)
# start timer

for row in no_nan_df['album']:
    al=row
    
#       trim album names of suffixes (e.g. .vol 1, [ep] etc.)
    al_trim = re.split(',|\[', al)

#       search for album ids 
    temp = spotify.search({"album":al_trim[0]}, search_type="album")
#     parse_album_ids = (temp["albums"]["items"][0]["id"])
    print(al)
    try:
        print(temp["albums"]["items"][0]["id"])
    except:
        print(temp["albums"]["href"])
#     try:
#         parse_album_ids = (temp["albums"]["items"][0]["id"])
# #   create dicitonary for albums not found
#     except:
# #         albums_not_found['artist'].append(ar)
#         albums_not_found['album'].append(al)
#     album_list.append(al)
#     list_ids.append(parse_album_ids)
#     album_ids.add(parse_album_ids)

# # return(get_album_tracks(album_ids))
# print(len(album_ids))
# print(len(list_ids))


Skeleton Tree


KeyError: 'albums'

In [14]:
# def search_for_albums(csv_path):
artists=[]
albums=[]

albums_not_found = {'artist': [], 'album': []}
album_ids = set()
album_list = []
list_ids = []

meta_16_df = pd.read_csv('albums_2016.csv')

no_nan_df = meta_16_df.dropna()
# no_nan_df.info()
#   initialize spotify client
spotify = SpotifyAPI(client_id, client_secret)
# start timer

# for row in no_nan_df['album']:
#     al=row
    
# #       trim album names of suffixes (e.g. .vol 1, [ep] etc.)
#     al_trim = re.split(',|\[', al)

# #       search for album ids 
print(spotify.search({"artist":"Frank Ocean"}, search_type="artist"))


{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3AFrank+Ocean&type=artist&offset=0&limit=20', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2h93pZq0e7k5yf4dywlkpM'}, 'followers': {'href': None, 'total': 8269073}, 'genres': ['alternative r&b', 'hip hop', 'lgbtq+ hip hop', 'neo soul', 'pop'], 'href': 'https://api.spotify.com/v1/artists/2h93pZq0e7k5yf4dywlkpM', 'id': '2h93pZq0e7k5yf4dywlkpM', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5eb981acc715c9138566885a1e7', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174981acc715c9138566885a1e7', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178981acc715c9138566885a1e7', 'width': 160}], 'name': 'Frank Ocean', 'popularity': 83, 'type': 'artist', 'uri': 'spotify:artist:2h93pZq0e7k5yf4dywlkpM'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/4RpWzrMf3lmcGxpyhOeBOc'}, 'followers': {'href': None, 'tot

In [57]:
no_nan_df['album_ids'] = list_ids

<ipython-input-57-cdaaed30383e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_nan_df['album_ids'] = list_ids


In [63]:
duplicateRowsDF = no_nan_df[no_nan_df.duplicated(subset=[ 'album_ids'], keep=False)]
duplicateRowsDF

,artist,album,date,week_num,meta_score,user_score,album_ids
19,Michael Kiwanuka,Love & Hate,"July 15, 2016",28,86,85,2Z9tU5T5Xad6u4rTGjZIy7
20,Maxwell,blackSUMMERS'night (2016),"July 1, 2016",26,86,84,2Z9tU5T5Xad6u4rTGjZIy7
50,Moodymann,DJ-Kicks,"February 19, 2016",7,83,78,4eMhrXXUCKHXIaG2xZGjjO
137,Cory Hanson,The Unborn Capitalist From Limbo,"November 11, 2016",45,81,0,21UK10eBA1kdKwRDON3q7H
138,Julia Jacklin,Don't Let the Kids Win,"October 7, 2016",40,81,79,21UK10eBA1kdKwRDON3q7H
141,Thalia Zedek,Eve,"August 19, 2016",33,81,0,0JGOiO34nwfUdDrD612dOp
206,Teenage Fanclub,Here,"September 9, 2016",36,79,78,0wEnZguPw6l4Rcdgzcm5qv
267,Jackmaster,DJ-Kicks,"July 8, 2016",27,78,0,4eMhrXXUCKHXIaG2xZGjjO
283,Bob Dylan,Fallen Angels,"May 20, 2016",20,77,72,3gl2RguatWgKSoCuN1tSw9
284,Childish Gambino,"""Awaken, My Love!""","December 2, 2016",48,77,85,3gl2RguatWgKSoCuN1tSw9


In [55]:
stars = []

for row in no_nan_df['artist']:
    stars.append(row)

slim_shadies = [x for n, x in enumerate(album_list) if x in album_list[:n]]
print(slim_shadies)

['DJ-Kicks', 'Here', 'IV']


In [56]:
albumed = []
for row in no_nan_df['album']:
    albumed.append(row)
    
duped = [x for n, x in enumerate(albumed) if x in albumed[:n]]
print(duped)

['DJ-Kicks', 'Here', 'IV']


['DJ-Kicks', 'Here', 'IV']


In [48]:

dupes = [x for n, x in enumerate(list_ids) if x in list_ids[:n]]
print(dupes)

['2Z9tU5T5Xad6u4rTGjZIy7', '21UK10eBA1kdKwRDON3q7H', '4eMhrXXUCKHXIaG2xZGjjO', '3gl2RguatWgKSoCuN1tSw9', '0JGOiO34nwfUdDrD612dOp', '0wEnZguPw6l4Rcdgzcm5qv', '1MGEMBqM3v8OUM2glv7MSx', '1zbH0DnpYxaoqTojXNLQb6', '3V4sro47Gv8FLSjQJO2PrR']


In [50]:
myList = [1,9,1,9,8,3,7,4]
samesies = [x for n, x in enumerate(myList) if x in myList[:n]]
print(samesies)

[1, 9]


In [24]:
print(len(set(list_ids)))

489


In [21]:
for albums in albums_not_found['album']:
    print(albums)

blackSUMMERS'night (2016)


In [43]:
if meta_16_df.duplicated(subset=['album'])is True:
    print(meta_16_df.duplicated['artist'])

In [30]:
no_nan_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 498
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artist      498 non-null    object
 1   album       498 non-null    object
 2   date        498 non-null    object
 3   week_num    498 non-null    int64 
 4   meta_score  498 non-null    int64 
 5   user_score  498 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 27.2+ KB


In [ ]:
def add_tracks_to_playlist(week_num, playlist_id):
    track_uris = [track for track in search_for_albums(week_num, csv_path)]
    track_limit = 100 
    # using list comprehension 
    batched_tracks = [track_uris[i * track_limit:(i + 1) * track_limit] for i in range((len(track_uris) + track_limit - 1) // track_limit)]  
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

    for batch in batched_tracks:

        request_data = json.dumps(batch)

        requests.post(
            url,
            data=request_data,
            headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'
            })
    return

In [ ]:
def create_playlist(week_num):
    print(week_num)
    # week_num = get_week_num()
    request_body = json.dumps({
        'name': f'2021-week {week_num} scrape',
        'description': f'metacritic rated albums for the {week_num}th of the year',
        'public': True
    })
    url = f'https://api.spotify.com/v1/users/{spotify_user_id}/playlists'


    response = requests.post(
        url,
        data = request_body,
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {playlist_token}'

    })
    response_json = response.json()
#     print(response_json)
    playlist_id = (response_json['id'])

    return add_tracks_to_playlist(week_num, playlist_id)